In [2]:
#paths

# path to the half of the dataset:  /import/c4dm-05/elona/Capitan
# path to the full dataset:  /import/c4dm-05/elona/Corpus
import os
print (os.environ['/data/home/acw507/.conda/envs/pytorchenv'])
whole_primus = '/data/scratch/acw507/Corpus/'

KeyError: '/data/home/acw507/.conda/envs/pytorchenv'

# Create the set of data to use

In [3]:
import json
import cv2
import numpy as np
import os
import sys
import tqdm
import glob

def loadPrimus(path, encoding, output_file):
    X=[]
    Y=[]
    limit = 80000
    i = 0 
    with open(output_file, "a") as f: # "a" is for appending instead of writing
        for folder in os.listdir(path):
            # get the image file name
            img_filename = f"{folder}.png"
            img_path = os.path.join(path, folder, img_filename)
            img = cv2.imread(img_path, 0)
            X.append(img)
            # print(img_path)

            # get the corresponding agnostic file name
            agnostic_filename = f"{folder}.agnostic"
            agnostic_file_path = os.path.join(path, folder, agnostic_filename)

            # read the contents of the agnostic file
            with open(agnostic_file_path, "r") as af:
                agnostic_contents = af.read()
                if encoding == "standard":
                    Y.append(agnostic_contents.strip().split("\t"))
                else:
                    splitted_seq = []
                    sequence = agnostic_contents.strip().split("\t")
                    for element in sequence:
                        for char in element.split("-"):
                            splitted_seq.append(char)
                    Y.append(splitted_seq)

            # write the image filename and the agnostic file contents to the output file
            f.write(f"{img_filename}\t")
            f.write(f"{agnostic_contents}\n")

            if i > limit:
                break
            i+=1




loadPrimus('/data/scratch/acw507/Corpus/', 'agnostic', '/data/home/acw507/music_recognition/data/only_shape/primus_whole_GT.txt')

ModuleNotFoundError: No module named 'cv2'

# CREATE A FILE WITH UNIQUE CLASSES

In [11]:
# this is just for the sake of having a file that lists all possible classes with position

import re 

def get_unique_classes(file_path):
    # Open the file and read in the contents
    with open(file_path, 'r') as file:
        contents = file.read()
        contents = re.sub(r'\S+\.png', '', contents)
    # Split the contents into words
   
    classes = contents.split()
    # Create a set of unique words
    unique_classes = set(classes)
    # remove number_only_words from unique_words
    # unique_words = [word for word in unique_words if not word.isnumeric()]
    unique_classes = sorted(unique_classes)
    # Write the unique words to a new .txt file
    with open('/homes/es314/deep-text-recognition-benchmark/new_primus_clean/unqiue_classes.txt', 'w') as outfile:
        for word in unique_classes:
            # remove the last three characters of the class
            # word = word[:-3]
            outfile.write(word + '\n')
    return unique_classes


unique_classes = get_unique_classes('/homes/es314/deep-text-recognition-benchmark/new_primus_clean/primus_whole_GT.txt')

# print(unique_words)


# CLEAN OUT POSITION OF NOTES

In [22]:
def get_words(file_path):
    # Open the file and read in the contents
    with open(file_path, 'r') as file:
        contents = file.read()
        
    # Split the contents into words
    words = contents.split()
    with open('/homes/es314/deep-text-recognition-benchmark/new_primus_clean/whole_clean_1.txt', 'w') as outfile:
        for word in words:
            if '/' in word:
                word = word.replace('/', '')
            if word.endswith(".png"):
                outfile.write('\n')
                word=word
            else:
                word = word[:-3]
                if '- ' in word:
                    word = word.replace('- ', ' ')
            outfile.write(word + ' ')

    return words

words = get_words('/homes/es314/deep-text-recognition-benchmark/new_primus_clean/whole_primus_GT.txt')


In [23]:
# produce another file with unique classes without positions

# this is just for the sake of having a file that lists all possible classes with position

import re 

def get_unique_classes(file_path):
    # Open the file and read in the contents
    with open(file_path, 'r') as file:
        contents = file.read()
        contents = re.sub(r'\S+\.png', '', contents)
    # Split the contents into words
   
    classes = contents.split()
    # Create a set of unique words
    unique_classes = set(classes)
    # remove number_only_words from unique_words
    # unique_words = [word for word in unique_words if not word.isnumeric()]
    unique_classes = sorted(unique_classes)
    # Write the unique words to a new .txt file
    with open('/homes/es314/deep-text-recognition-benchmark/new_primus_clean/no_position_unique_classes.txt', 'w') as outfile:
        for word in unique_classes:
            # remove the last three characters of the class
            # word = word[:-3]
            outfile.write(word + '\n')
    return unique_classes


unique_classes = get_unique_classes('/homes/es314/deep-text-recognition-benchmark/new_primus_clean/whole_clean_1.txt')

# CREATE MAPPING DICTIONARY

In [32]:
import json

# Read the mapping from json file
with open("/homes/es314/deep-text-recognition-benchmark/new_primus_clean/mapping.json") as f:
    mapping = json.load(f)

# Read the contents of test.txt
with open("/homes/es314/deep-text-recognition-benchmark/new_primus_clean/whole_clean_1.txt") as f:
    content = f.read()

# Replace each symbol with the corresponding mapping
for symbol, replacement in mapping.items():
    content = content.replace(symbol, replacement)

# Write the modified content to a new file
with open("primus_half_mapped.txt", "w") as f:
    f.write(content)

In [6]:
# the dict

import json

# read the input file
with open("/homes/es314/deep-text-recognition-benchmark/new_primus_clean/no_position_unique_classes_whole.txt", "r") as f:
    values = f.read().split("\n")

# create the dictionary
encoding = {}
characters = "abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789!#$%&"
for i, value in enumerate(values):
    encoding[value] = characters[i % len(characters)]

# write the dictionary to output file
# with open("output.txt", "w") as f:
    json.dump(encoding, f)

ValueError: I/O operation on closed file.